# Recursion and memoization
Today, we will talk about a really neat way to speed up recursive algorithms, called memoization.

In [1]:
%config InteractiveShell.ast_node_interactivity="none"

In [2]:
!wget https://raw.githubusercontent.com/jamcoders/syllabus-resources-2023/main/week3/lecs/boaz_utils.ipynb
%run "boaz_utils.ipynb"

# Slow fibonacci

Recall the fibonacci function you've seen before:

In [27]:
def fib(n):
    '''Slow fibonacci computation'''
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

for i in range(8):
    print(f"fib({i}) = {fib(i)}")

fib(0) = 0
fib(1) = 1
fib(2) = 1
fib(3) = 2
fib(4) = 3
fib(5) = 5
fib(6) = 8
fib(7) = 13


In [28]:
print(fib(34)) # that's kinda slow

5702887


In [29]:
# This cell would take centuries to run! 
print(fib(100))  
# Try running it but press Ctrl+C or the stop button to abort

KeyboardInterrupt: 

Observe that a recursive call is done every time we need the result of `fib(n)` for $n \geq 2$.  Let's see just how many of those calls are being made for various values of $n$:  

In [30]:
def cfib(n):
    def fib(n, counts):
        """Fibonacci where we count how many calls were made"""
        if n in counts:
            counts[n] += 1
        else: # not in dictionary, initialize to 1
            counts[n] = 1
        if n < 2:
            return n
        return fib(n-1, counts) + fib(n-2, counts)

    call_counts = {}  # n -> # times fib(n) called
    result = fib(n, call_counts)
    print(call_counts)
    return result

print(cfib(32))


{32: 1, 31: 1, 30: 2, 29: 3, 28: 5, 27: 8, 26: 13, 25: 21, 24: 34, 23: 55, 22: 89, 21: 144, 20: 233, 19: 377, 18: 610, 17: 987, 16: 1597, 15: 2584, 14: 4181, 13: 6765, 12: 10946, 11: 17711, 10: 28657, 9: 46368, 8: 75025, 7: 121393, 6: 196418, 5: 317811, 4: 514229, 3: 832040, 2: 1346269, 1: 2178309, 0: 1346269}
2178309


In computing `fib(32)`, the computer recomputed `fib(30)` twice and `fib(0)` more than 1 million times (1,346,269 times to be precise)! How silly!
That is happening because `fib(32)` calls `fib(30)` directly and `fib(31)` which calls `fib(30)` *again*, and then they all  call `fib(29)`. (Curious how the number of calls looks like a Fibonacci sequence itself.) 

Importantly, each call to `fib` always returns the same answer for the same input. We know this because of how we implemented `fib`, but the computer doesn't know that ahead of time, it executes `fib` for each call because we told it to do that.

This suggests a nice simple idea to improve the performance of the code. 



# Memoize
Instead of just counting how many calls we make, let's save time and use our dictionary to remember the answers. That way, we never have to call `fib` more than once on the same input! 

Were's the "r"? Whoeve inveted memoization needs to lean to spell bette.

In [31]:
fib_memory = {}   # using "global variables" like this is not a great programming practice

def fib_memo(n):
    '''Compute fibonacci number if not already in memory'''
    if n in fib_memory:  
        return fib_memory[n]
    if n < 2:
        ans = n
    else:
        ans = fib_memo(n - 1) + fib_memo(n - 2)
    fib_memory[n] = ans
    return ans
    
print(fib_memo(10))
print(fib_memory)

55
{1: 1, 0: 0, 2: 1, 3: 2, 4: 3, 5: 5, 6: 8, 7: 13, 8: 21, 9: 34, 10: 55}


In [32]:
print(fib_memo(100)) # no sweat!

354224848179261915075


In [46]:
print(fib_memo(1000))

43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875


# Automatic memoization!

In python we can automate this step using a function that takes a function as input and returns a new function as 
output. That may sound crazy, but it works! We will create a memoize function that remembers the answers
for *any* function.

Save this code somewhere in case you want to use it later.

In [35]:
def memoize(func):
    '''Remember the values of a function func'''
    memory = {}  # the memory dictionary -- this is a better style than a global variable 
    def helper(*args):
            key = str(args)  # convert the function inputs to a string so it can work on any input
            if key in memory:
                return memory[key]
            ans = func(*args)
            memory[key] = ans 
            return ans
    helper.memory = memory  # so we can access it later
    return helper

In [36]:
# It's easy to create a memoized version of the fibonacci function.
# We can use it to speed up the slow recurisve fib function above.
fib = memoize(fib) 

# now let's try again
print(fib(100))

354224848179261915075


In [37]:
# run this if you want to see the contents of the memory
print(fib.memory)

{'(1,)': 1, '(0,)': 0, '(2,)': 1, '(3,)': 2, '(4,)': 3, '(5,)': 5, '(6,)': 8, '(7,)': 13, '(8,)': 21, '(9,)': 34, '(10,)': 55, '(11,)': 89, '(12,)': 144, '(13,)': 233, '(14,)': 377, '(15,)': 610, '(16,)': 987, '(17,)': 1597, '(18,)': 2584, '(19,)': 4181, '(20,)': 6765, '(21,)': 10946, '(22,)': 17711, '(23,)': 28657, '(24,)': 46368, '(25,)': 75025, '(26,)': 121393, '(27,)': 196418, '(28,)': 317811, '(29,)': 514229, '(30,)': 832040, '(31,)': 1346269, '(32,)': 2178309, '(33,)': 3524578, '(34,)': 5702887, '(35,)': 9227465, '(36,)': 14930352, '(37,)': 24157817, '(38,)': 39088169, '(39,)': 63245986, '(40,)': 102334155, '(41,)': 165580141, '(42,)': 267914296, '(43,)': 433494437, '(44,)': 701408733, '(45,)': 1134903170, '(46,)': 1836311903, '(47,)': 2971215073, '(48,)': 4807526976, '(49,)': 7778742049, '(50,)': 12586269025, '(51,)': 20365011074, '(52,)': 32951280099, '(53,)': 53316291173, '(54,)': 86267571272, '(55,)': 139583862445, '(56,)': 225851433717, '(57,)': 365435296162, '(58,)': 591286

# Decorative Memoization

Python provides a shortcut to defining `fib` and `fib = memoize(fib)` in one step. This is what is illustrated below.


In [18]:

@memoize
def pretty_fib(n):
    '''Fibonacci number'''
    if n < 2:
        return n
    return pretty_fib(n-1) + pretty_fib(n-2)


In [38]:
print(pretty_fib(100))  # fast, and no need to create a separate dictionary

354224848179261915075


## Binomial Coefficients
You are probably already aware of the notation $n \choose r$ to represent the number of ways to choose $r$ items from a collection of $n$ of them.

You've probably also been given the following formula:
$$ {{n} \choose {r}} = \frac{n!}{r! (n-r)!} $$

Let us discuss a more intuitive way to compute this value.

Consider item 1 of the $n$ items (it could have been any of them).  
 - Either it is included in the selected $r$ or it is not.  
 - Those two possibilities are mutually exclusive
 
 So, the value of $n \choose r$ must be the sum of the number of ways to attain either outcome.
 
 - The number of ways to include item 1 would be $n-1 \choose r-1$
 - The number of ways to not to do that would be $n-1 \choose r$
 
 We can use this idea to implement a recursive solution to computing $n \choose r$. 
 
 What are suitable base cases though?

In [39]:
def choose(n, r):
    if n < r:
        return 0
    elif r == 0:
        return 1
    else:
        return choose(n-1, r) + choose(n-1, r-1)

assert choose(4, 2) == 6
assert choose(5, 3) == 10
assert choose(4, 0) == 1
assert choose(0, 0) == 1

In [40]:
print(choose(50, 10))

KeyboardInterrupt: 

In [41]:
@memoize
def mchoose(n, r):
    if n < r:
        return 0
    elif r == 0:
        return 1
    else:
        return mchoose(n-1, r) + mchoose(n-1, r-1)

In [44]:
print(mchoose(0, 10))

100891344545564193334812497256
